In [1]:
#KS200, 기본 코드, overfitting, Train_test_split, cv = TimeSeriesSplit

import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import TimeSeriesSplit, cross_val_score, train_test_split
from sklearn.model_selection import RandomizedSearchCV

from sklearn.metrics import confusion_matrix
import joblib

# 데이터 불러오기 및 전처리
model_data = pd.read_csv('../main/ks200.csv', encoding='utf-8')

# Label Encoding
from sklearn.preprocessing import LabelEncoder
encoder = LabelEncoder()
model_data['레이블'] = encoder.fit_transform(model_data['레이블'])

# 날짜 처리 및 정렬
model_data['날짜'] = pd.to_datetime(model_data['날짜'])
model_data.set_index('날짜', inplace=True)
model_data.sort_index(inplace=True)

# 특성과 레이블 분리
X = model_data.drop(columns=['레이블'])
y = model_data['레이블']

# 데이터 분할
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

# 모델 초기화 및 하이퍼파라미터 그리드 정의
rnd_clf = RandomForestClassifier(n_estimators=100, n_jobs=1, random_state=42)
param_dist_rf = {
    'n_estimators': [50, 100, 500],
    'max_leaf_nodes': [20, 30, 40, 50],
    'max_features': [1, 2, 3],
    'max_depth': [5, 10, 15],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}

# 하이퍼파라미터 튜닝
time_split_cv = TimeSeriesSplit(n_splits=10)
rnd_search = RandomizedSearchCV(rnd_clf, param_dist_rf, cv=time_split_cv, random_state =42)
rnd_search.fit(X_train, y_train)

# 최적 모델 선택 및 훈련
best_clf = rnd_search.best_estimator_
best_clf.fit(X_train, y_train)

# 모델 평가
train_score = best_clf.score(X_train, y_train)
test_score = best_clf.score(X_test, y_test)

# 10-fold cross-validation의 정확도 계산
cv_scores = cross_val_score(best_clf, X_train, y_train, cv=time_split_cv, scoring='accuracy')
cv_mean_score = cv_scores.mean()

# label encoding된 값 찾기
up = encoder.transform(['up'])[0]
neutral = encoder.transform(['neutral'])[0]
down = encoder.transform(['down'])[0]

# 혼동 행렬 확인
y_test_pred = best_clf.predict(X_test)
cm_test = confusion_matrix(y_test, y_test_pred, labels=[up, neutral, down])
y_all_pred = best_clf.predict(X)
cm_all = confusion_matrix(y, y_all_pred, labels=[up, neutral, down])


# 특성 중요도 확인
feature_importance = list(zip(X_train.columns, best_clf.feature_importances_))

# 모델 저장
joblib.dump(best_clf, "separation22.pkl")

# 10. backtesting용 과거의 예측데이터 생성
y_prediction = rnd_clf.predict(X) 
y_pred = pd.Series(y_prediction, index=y.index) 

# 결과 출력

# 결과를 출력합니다.
print(f"{rnd_search.best_params_}\n")
print("<10-fold cross-validation>")
print("accuracy score mean: ", cv_mean_score)
print("\n<AI model: machine learning done >")
print("accuracy_score of train data(0.8 of sample): ", train_score)
print("accuracy_score of test data(0.2 of sample): ", test_score)
print("\n<Confusion matrix>")
print("(of test)")
print("up", "neutral", "down")
print(cm_test)
print("(of all)")
print("up", "neutral", "down")
print(cm_all)
# <Feature importance>를 내림차순으로 정렬하여 출력
print("\n<Feature importance>")
sorted_feature_importance = sorted(feature_importance, key=lambda x: x[1], reverse=True)
for name, score in sorted_feature_importance:
    print(name, ": ", score)

print("\n< AI model: save >")

ImportError: C extension: None not built. If you want to import pandas from the source directory, you may need to run 'python setup.py build_ext' to build the C extensions first.

In [12]:
# -*- coding: utf-8 -*- 
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt 
# -1. MDD 함수 정의
def MDD(list_values): 
 mdd_value = 0 
 for i in range(1, len(list_values)): 
    bw_max = max(list_values[:i]) 
    curr = list_values[i] 
    mdd = curr / bw_max - 1 
 if mdd < mdd_value: 
    mdd_value = mdd 
 return mdd_value 
# 0. 사후방향성 클래스를 수치로 전환하는 함수 정의 (up=1, neutral=0, down=-1) 
def convert_num(pred): 
    pred_num = np.empty(len(pred)) 
    pred_num[pred=='up']=1 
    pred_num[pred=='neutral']=0 
    pred_num[pred=='down']=-1 
    pred_num[pred.isna()]=np.NaN 
    
    return pred_num 
# 1. 월간 시장수익률 데이터 가져오기
# 데이터는 일자/월수익률(원수치)/코스피지수/월말하루전영업일로 구성
kdata = pd.read_csv("kospi_month_end.csv")
# 2. AI모델로 예측한 예측정보를 가져오기
# 기존 AI모델 학습 프로세스에서 만들어낸 y_pred 데이터를 조회함. 
# 매월말에 예측했던 투자의견을 가져옴. 단, 실제 투자를 위해서, 월말하루전영업일 기준 자료를 가져옴
# kdata['stage']='' 
kdata['stage'] = y_pred #말일자 아님
for index in kdata.index: 
   kdata.loc[index,'stage']=y_pred[kdata.loc[index,'before_last']] 
# 3. 전월말 투자의견 열 생성
kdata['pre_stage']= kdata['stage'].shift(1) 
kdata['port_return']=0 
# 4. 전략 수익률 생성
# 전월말 투자의견이 상승이면 코스피 long, 보합이면 Cash, 하락이면 코스피 Short 실행. 
# 해당 전략에 따라 포트 월별수익률(port_return) 생성
kdata.loc[kdata['pre_stage']=='up', 'port_return'] = kdata['m_return']*1 
kdata.loc[kdata['pre_stage'].isna(), 'port_return'] = kdata['m_return']*1 
kdata.loc[kdata['pre_stage']=='neutral', 'port_return'] = 0 
kdata.loc[kdata['pre_stage']=='down', 'port_return'] = kdata['m_return']*-1 
# 코스피와 모델포트폴리오의 누적수익률(1에서 시작하는 인덱스 형태) 생성
kdata['kospi_cumul']=(1+kdata['m_return']).cumprod() 
kdata['port_cumul']=(1+kdata['port_return']).cumprod() 
# 5. 백테스팅 결과 기록(CAGR, 변동성, Sharpe ratio, MDD) 
my_back = {'months':len(kdata)} 

my_back['k_cumul_return_idx']=kdata['kospi_cumul'][-1] 
my_back['k_cumul_return_pct']=(my_back['k_cumul_return_idx']-1)*100 
my_back['k_cagr']=(my_back['k_cumul_return_idx']**(12/my_back['months']))-1 
my_back['k_cagr_pct']=my_back['k_cagr']*100 
my_back['k_vol_pct']=np.std(kdata['m_return'])*np.sqrt(12)*100 
my_back['k_Sharpe']=my_back['k_cagr_pct']/my_back['k_vol_pct'] 
my_back['k_MDD']=MDD(kdata['kospi_cumul'])*100 

my_back['port_cumul_return_idx']=kdata['port_cumul'][-1] 
my_back['port_cumul_return_pct']=(my_back['port_cumul_return_idx']-1)*100 
my_back['port_cagr']=(my_back['port_cumul_return_idx']**(12/my_back['months']))-1 
my_back['port_cagr_pct']=my_back['port_cagr']*100 
my_back['port_vol_pct']=np.std(kdata['port_return'])*np.sqrt(12)*100 
my_back['port__Sharpe']=my_back['port_cagr_pct']/my_back['port_vol_pct'] 
my_back['port_MDD']=MDD(kdata['port_cumul'])*100 

# 6. 백테스팅 결과 출력하기
print("<Backtesting result>") 
for key, value in my_back.items(): 
 print("{:22}: {:>8.3f}".format(key, value)) 
 
# 포트폴리오 누적수익률 그래프
kdata['port_cumul'].plot() 
plt.title('Portfolio performance index') 
plt.ylabel('\'02/12/31 = 1') 
plt.show() 

# 7. 월말 모델전망치와 실제결과치 출력
# 월말 실제결과치 입력
for index in kdata.index: 

 kdata.loc[index,'real_stage']=y[kdata.loc[index,'before_last']] 
# 사후방향성 클래스를 수치로 변환
kdata['stage_num']=convert_num(kdata['stage']) 
kdata['real_stage_num']=convert_num(kdata['real_stage']) 

# 전망치와 결과치의 그래프 출력
kdata.plot(y=['stage_num', 'real_stage_num'], label=['model forecast','real direction']) 
plt.title('Model forecast vs. Real direction') 
plt.ylabel('up=1, neutral=0, down=-1') 
plt.show() 

ValueError: Length of values (585) does not match length of index (150)